# Lockdrop Analysis v2

## 0. Imports

In [3]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import json
from collections import defaultdict

## 1. Read Data

In [4]:
TX_HASH = 'transactionHash'
ETH_ADDRESS = 'ethAddress'
LOCKED_ETH = 'lockedEth'
LOCK_DURATION = 'lockDuration'
PLASM_ADDRESS = 'plasmAddress'
PLM_BALANCE = 'plmBalance'

with open('lockdrop-participant-balances.json') as ld_participant_balances_file:
    participant_balances = json.load(ld_participant_balances_file)

In [5]:
participant_balances[0]

{'transactionHash': '0xdfeae5962ad6933df1e195a7d1db220f08e56996d9445afb947dc6b32bbee62b',
 'ethAddress': '0x0dEf9b4811E4c4E9Da5ACF82d421C97B5E9C152F',
 'lockedEth': '1.23',
 'lockDuration': 1000,
 'plasmAddress': 'WrTbSfrx11QMGwfn8GbmErX5PbsSEpEm1akkeyzbUNCF3Cs',
 'plmBalance': '295476019650980300000'}

In [6]:
# IGNORE PARTICIPAN BALANCES

In [7]:
ETH = 'eth'
DURATION = 'duration'
LOCK = 'lock'
INTRODUCER = 'introducer'
BLOCK_NO = 'blockNo'
TIMESTAMP = 'timestamp'
LOCK_OWNER = 'lockOwner'  # this appears to be eth address

In [8]:
with open('lockdrop-first-event.json') as ld_first_event_file:
    first_event = json.load(ld_first_event_file)
    
with open('lockdrop-second-event.json') as ld_second_event_file:
    second_event = json.load(ld_second_event_file)

In [9]:
first_event[0]

{'eth': '1230000000000000000',
 'duration': 1000,
 'lock': '0x757b339A5e14323900159b4F06407fD9126e7b36',
 'introducer': '0x0000000000000000000000000000000000000000',
 'blockNo': 9672720,
 'timestamp': 1584230749,
 'lockOwner': '0x0dEf9b4811E4c4E9Da5ACF82d421C97B5E9C152F',
 'transactionHash': '0xdfeae5962ad6933df1e195a7d1db220f08e56996d9445afb947dc6b32bbee62b'}

In [10]:
second_event[0]

{'eth': '2000000000000000000',
 'duration': 30,
 'lock': '0x19B2E1f82503c7C20e58BFce3344206c61E23366',
 'introducer': '0x0000000000000000000000000000000000000000',
 'blockNo': 10765336,
 'timestamp': 1598832035,
 'lockOwner': '0xa04c091e9FD824F86Efd27F0d40Fea4240115c82',
 'transactionHash': '0x2544b978b0004b21b16753a177f54dab6e7bbbb054909837390c009a3e132ef8'}

In [11]:
len(first_event), len(second_event), len(participant_balances)

(368, 12021, 4759)

## 2. Aggregate First & Second Lockdrop Events

In [12]:
len(first_event + second_event)

12389

In [13]:
new_aggregate = defaultdict(int)

first_event_aggregate = defaultdict(int)
second_event_aggregate = defaultdict(int)

for x in first_event:
    first_event_aggregate[x[LOCK_OWNER].lower()] = max(x[DURATION], first_event_aggregate[x[LOCK_OWNER].lower()])
    
for x in second_event:
    second_event_aggregate[x[LOCK_OWNER].lower()] = max(x[DURATION], second_event_aggregate[x[LOCK_OWNER].lower()])
    
new_aggregate = first_event_aggregate | second_event_aggregate

In [14]:
len(first_event_aggregate), len(second_event_aggregate)

(262, 6042)

In [15]:
len(new_aggregate)

6234

## 3. ETH-Astar mapping

In [19]:
with open('first-lockdrop.json') as first_ld_file:
    first_ld_claim = json.load(first_ld_file)

first_eth_to_plasm = {x['ethAddress'].lower(): x['plasmAddress'] for x in first_ld_claim}

In [20]:
len(first_ld_claim), len(first_eth_to_plasm)

(368, 262)

In [21]:
with open('second-lockdrop.json') as second_ld_file:
    second_ld_claim = json.load(second_ld_file)
    
second_eth_to_plasm = {x['ethereumAddress'].lower(): x['plasmAddress'] for x in second_ld_claim}

In [22]:
len(second_ld_claim), len(second_eth_to_plasm)

(8925, 5789)

In [23]:
eth_to_plasm = first_eth_to_plasm | second_eth_to_plasm

In [24]:
len(eth_to_plasm)

5985

In [25]:
len(new_aggregate)

6234

In [26]:
len(new_aggregate.keys() - eth_to_plasm.keys())

249

In [27]:
plasm_to_duration = dict()

for (eth_addr, duration) in new_aggregate.items():
    if eth_addr not in eth_to_plasm.keys():
        print(f"No matching PLASM address for {eth_addr} ({duration})")
    else:
       plasm_to_duration[eth_to_plasm[eth_addr]] = duration 

No matching PLASM address for 0x9e2178376af13dbb3d939fa62a8d33bf74912e3d (100)
No matching PLASM address for 0x11ba6c4732b1a7f30dea51c23b8ed4c1f88dcd57 (100)
No matching PLASM address for 0xb9b09311ea697c9bd16b9c41f759b852cc14b169 (1000)
No matching PLASM address for 0x0d4ddea3849c54e16c85cb4354943dc876e05552 (1000)
No matching PLASM address for 0xfce11e5b8940889bed838788610aa05c87f2fbd8 (100)
No matching PLASM address for 0xd46c10d4f93c68462a78e2d7c38cdb16d3d81495 (100)
No matching PLASM address for 0x502096777a0f1357c5ac9b7cfc9c929ec77bff6d (100)
No matching PLASM address for 0x7b08336f515608024409a7007e93ae96531dd1d7 (1000)
No matching PLASM address for 0x055ad5e56c11c0ef55818155c69ed9ba2f4b3e90 (30)
No matching PLASM address for 0xfeeff524f5564f3444b1ef142c6d3cdc95b3755e (300)
No matching PLASM address for 0xf3540576baac4524de3bb4f7b3b008f849e188bd (1000)
No matching PLASM address for 0x8fc1dcc17fb7d5308fc0b0f361f65bcf9d7f3087 (1000)
No matching PLASM address for 0x0486a831d78fa119

In [28]:
len(plasm_to_duration), len(new_aggregate), len(eth_to_plasm)

(5985, 6234, 5985)

In [30]:
with open('plasm_to_duration.json', 'w') as ptd:
    json.dump(plasm_to_duration, ptd)